### Drying Model

We model a continuous conveyor belt dryer. The solid placed on the conveyor is dried as it passes through each oven zone. Air is used as the drying medium and flows perpendicular to the direction of the product, with the direction being reversed in subsequent zones. The temperature and velocity of the air are controlled independently for each zone. A schematic of the oven is shown in figure 

<img src = "images/oven_schematic.png" width = 1400 height = 400 />

We build mass (moisture) and energy (temperature) balance equations for the product and the air flowing across the oven.

#### 1. Solid Model

We treat the product as a collection of small solid particles and moisture. For each solid particle, we model the rate of change of moisture content and the rate of change of temperature as follows

We model the rate of change of moisture content inside the solid particle and rate of change of temperature of the solid as follows
\begin{align*}\tag{1}
\begin{split}
\text{Accumulation} & = \text{In} - \text{Out } + \text{Generation} \\\\
\rho V_p \frac{dX}{dt} & = -
\begin{cases}
    A_p k_m \left[ (\rho Y_e)_{air} - (\rho Y)_{air} \right] & \text{if } X > X_c \\
    A_p k_m \left[ (\rho Y_e)_{air} - (\rho Y)_{air} \right] \left( \frac{X - X_e}{X_c - X_e} \right) & \text{otherwise} \\
\end{cases}\\\\
\rho V_p c_p \frac{dT}{dt} & = 
\begin{cases}
    A_p h \left[ T_{air} - T \right] - A_p h_{vap} k_m  \left[ (\rho Y_e)_{air} - (\rho Y)_{air} \right] = 0 & \text{if } X > X_c \\
    A_p h \left[ T_{air} - T \right] - A_p h_{vap} k_m  \left[ (\rho Y_e)_{air} - (\rho Y)_{air} \right] \left( \frac{X - X_e}{X_c - X_e} \right) & \text{otherwise}  \\ 
\end{cases}
\end{split}
\end{align*}

where X is the moisture content in the solid dry basis, $V_p$ is the volume of the particle, $A_P$ is the surface area of the particle, $k_m$ is the mass transfer coefficient, $Y_e$ is the equilibrium moisture content at temperature $T$, $Y$ is the moisture content in air dry basis, $X_c$ is the critical moisture content, $X_e$ is the equilibrium moisture content, $\rho$ is the density of solid, $\rho _{air} $ is the density of air, $h$ is the heat transfer coefficient, $h_{vap}$ is the enthalpy of vaporization, $T_{air}$ is the temperature of air and $c_p$ is the specific heat capacity, $t$ is the residence time of solid inside the oven.

#### 2. Air Model
We model the rate of change of moisture content in air dry basis and rate of change of temperature of air as follows

\begin{align*}\tag{2}
\begin{split}
\text{Accumulation} & = \text{In} - \text{Out } + \text{Generation} \\\\
v \rho _{air} V_p \frac{dY}{dy} & = 
\begin{cases}
    (1 - \epsilon) A_p k_m \left[ (\rho Y_e)_{air} - (\rho Y)_{air} \right] & \text{if } X > X_c \\
    (1 - \epsilon) A_p k_m \left[ (\rho Y_e)_{air} - (\rho Y)_{air} \right] \left( \frac{X - X_e}{X_c - X_e} \right) & \text{otherwise} \\
\end{cases}\\\\
v \rho _{air} V_p c_{p_{air}} \frac{dT_{air}}{dy} & = 
\begin{cases}
    (1 - \epsilon) A_p \left[ h \left[ T_{air} - T \right] - h_{vap} k_m  \left[ (\rho Y_e)_{air} - (\rho Y)_{air} \right] \right] = 0 & \text{if } X > X_c \\
    (1 - \epsilon) A_p \left[ h \left[ T_{air} - T \right] - h_{vap} k_m  \left[ (\rho Y_e)_{air} - (\rho Y)_{air} \right] \left( \frac{X - X_e}{X_c - X_e} \right) \right] & \text{otherwise}  \\ 
\end{cases}
\end{split}
\end{align*}

where $\rho _{air}$ is the density of air, $c_{p_{air}}$ is the heat capacity of air, $v$ is the velocity of air $y$ is the height of the product, and $\epsilon$ is bed porosity defined as 

\begin{equation*}\tag{3}
    \epsilon = 1 - \frac{\rho _{B}}{\rho _{p}}
\end{equation*}

$\rho _B$ is the bulk density and $ \rho _p$ is the density of the solid particle. The following equation gives the heat and mass transfer equations 

\begin{align}\tag{4}
\begin{split}
    k_m & = 
    \begin{cases}
        k_0 \log{\frac{1 + \mu Y_e}{1 + \mu Y}}& \text{if } X > X_c \\
        k_{\phi}(X, Y, T, T_{air}) & \text{otherwise}
    \end{cases}\\
    h & = h_{\xi}(X, Y, T, T_{air})
\end{split}
\end{align}

where $k_0$ is a multiplicative constant, $\mu$ is the ratio of molecular weight of air to molecular weight of water, and $k_m$ is the mass transfer coefficient. The mass transfer coefficient is given by a correlation when the moisture is above the critical moisture rate and by a neural network $k_{\phi}$, parameterized by $\phi$, which takes input as the states at the current position in the solid. Heat transfer coefficient is also given by a neural network $h_{\xi}$ parameterized by $\xi$. 

#### 3. Simulating the drying equations

Equation 1 and equation 2 combined form a partial differential equation with the moisture and temperature of the solid changing in the vertical and the horizontal direction. We solve this system using method of lines, in which we discretize equation 2 using finite difference techniques, substitute the values of moisture and temperature of the air in equation 1, and then use an ode solver to solve the resulting equation. A pseudo-code is given in algorithm 1

<img src = "images/algo_fwd_sim.png" width = 1000 height = 400 />


#### 4. Recirculation 

To save energy, the exhaust air from the oven is recycled, as shown in figure \ref{fig:recirculation}. The amount of recycled air is governed by the recirculation ratio shown in equation 4. Since there is no mechanism to control the moisture of the air as it is heated in the burner, the moisture content in the ovens inlet air depends on the moisture content in the exhaust. This requires solving a root-finding problem for the inlet moisture content of the air. 

\begin{equation}\tag{5}
    \text{recirculation ratio} = \frac{\text{Amount of exhaust recycled}}{\text{Total exhaust}}
\end{equation}

<br>

<table><tr>
<td><img src = "images/recirculation.png" width = 450 height = 500 /></td>
<td><img src = "images/algo_recir.png" width = 1000 height = 400 /></td>
</tr></table>


In [ ]:
import os
import operator

import jax.numpy as jnp
from jax import tree_util
import pickle

from oven import oven_dynamics
from oven.phy_props import _mass_transfer_coefficient, _heat_transfer_coefficient, moisture_content
from oven.scaling import choose_scaling
from oven.data_structures import Constants, SimulationData, OdeKwargs, Parameters, Controls
from oven import plot
from oven.utils import get_coefficients

In [ ]:
nzones = 2 # number of zones
reverse_zone = 1 # zone from which the direction of flow reverses (goes bottom to top)

(moisture_max, moisture_min, temperature_max, temperature_min), (scale_states, unscale_states, _, _) = choose_scaling("max") # choose how to scale states

def scaled_moisture_content(T : jnp.ndarray, density_air : float, relative_humidity : float = 1) -> jnp.ndarray :
    """ Function that gives moisture content """
    _moisture = moisture_content(unscale_states(T, temperature_max, temperature_min), density_air, relative_humidity)
    return scale_states(_moisture, moisture_max, moisture_min)

def scaled_mass_transfer_coefficient(x : jnp.ndarray, p : jnp.ndarray, c : Constants) -> jnp.ndarray :
    """ Function that gives mass transfer coefficient """
    return _mass_transfer_coefficient(x, p, c)

def scaled_heat_transfer_coefficient(x : jnp.ndarray, p : jnp.ndarray, c : Constants) -> jnp.ndarray :
    """ Function that gives heat transfer coefficient """
    return _heat_transfer_coefficient(x, p, c)

constants = Constants(
    radius_particle = 4e-6,
    voidage = 0.99,
    enthalpy_vaporization_water = 2.e6,
    specific_heat_capacity_air = 1047.,
    product_height = 0.1,
    density_particle = 2500.,
    specific_heat_capacity_solid = 800.,
    equilibrium_moisture = scale_states(0.008, moisture_max, moisture_min), # equilibrium moisture content in solid

    moisture_max = moisture_max, 
    moisture_min = moisture_min,
    temperature_max = temperature_max,
    temperature_min = temperature_min,

    ny = 20, # number of discretization points in the vertical direction
    nx = 20, # number of discretization points in the horizontal direction
    ntimes = 12 + 1, # number of measurements for given residence time + initial condition

    ode_kwargs = OdeKwargs(rtol = 1e-6, atol = 1e-6, mxstep = 10_000), # ode solver options
    
    # Custom functions
    scaled_moisture_content = scaled_moisture_content, 
    scaled_mtc = scaled_mass_transfer_coefficient,
    scaled_htc = scaled_heat_transfer_coefficient,
    density_air = lambda *args : 1,
)

controls = Controls(inputs = 
    {
        "init_temperature_air" : scale_states(jnp.array([500.]), temperature_max, temperature_min), # setpoint - line loss 
        "init_velocity_air" : jnp.array([0.098]),
        "init_moisture_air" : scale_states(jnp.array([0.01]), moisture_max, moisture_min),
        "residence_time" : jnp.array([12.]),
        "constant" : jnp.array([1.]),
        "recirculation_ratio" : jnp.array([0.5]), # must be between 0 to 1
    }
)

assert tree_util.tree_reduce(operator.add, tree_util.tree_map(lambda x : jnp.logical_not(len(x) == 1 or len(x) == nzones), controls)) > 0, "Controls length can either be 1 or nzones"

ny = constants.ny
ntimes = constants.ntimes
nx = constants.nx
solid_moisture_init = scale_states(jnp.array([0.08]), moisture_max, moisture_min) # scaled initial moisture in solid across length
solid_temperature_init =  scale_states(jnp.array([331.]), temperature_max, temperature_min) # scaled initial temperature in solid across length
_xinit = jnp.concatenate((solid_moisture_init, solid_temperature_init))
xinit = jnp.tile(_xinit, (ny, 1))

time_span = jnp.arange(controls["residence_time"][0]*nzones).flatten()
height_span = jnp.linspace(0, constants.product_height, ny)

# load parameters from directory
load_dir = "log/R13SSR1/2024-04-29 16:59:19.419159" # log directory should be in path
assert os.path.exists(load_dir), f"Path {load_dir} does not exist"
with open(f"{load_dir}/saved_params/params", "rb") as file:
    meta_data = pickle.load(file)
    parameters = meta_data["opt_params"]

"""
parameters = Parameters(params = 
    {
        "heat_transfer_coefficient" : parameters_init(key, [4, 2, 1]),
        "mass_transfer_coefficient_falling" : parameters_init(key, [4, 2, 1]), # mass transfer coefficient X enthalpy of vaporization
        "mass_transfer_coefficient_constant" : jnp.array([.5]), # mass transfer coefficient X enthalpy of vaporization
        "constant_jump" : jnp.array([5.]),
        "critical_moisture" : scale_states(jnp.array([.07]), moisture_max, moisture_min),
    }
)"""

solution = (
    moisture_solid, temperature_solid, moisture_air, 
    temperature_air, ts_event) = oven_dynamics(
                xinit, parameters, 
                controls, constants, reverse_zone, nzones, recirculation = False
    )

In [ ]:
solution = (moisture_solid, temperature_solid, moisture_air, temperature_air, ts_event) = tree_util.tree_map(jnp.vstack, solution)

assert tree_util.tree_reduce(operator.add, tree_util.tree_map(lambda z : jnp.isnan(z).any(), solution)) > 0, "Nan detected"

htc, mtc = get_coefficients(solution[:-1], parameters, constants, scaled_moisture_content, scaled_mass_transfer_coefficient, scaled_heat_transfer_coefficient)

# scale back states
moisture_solid, moisture_air = tree_util.tree_map(lambda z : unscale_states(z, moisture_max, moisture_min), (moisture_solid, moisture_air))
temperature_solid, temperature_air = tree_util.tree_map(lambda z : unscale_states(z, temperature_max, temperature_min), (temperature_solid, temperature_air))

plot(
    SimulationData(moisture_solid, temperature_solid, moisture_air, temperature_air, htc, mtc),
    (time_span, height_span, 1, controls["residence_time"]),
    None, # provide a dictionary with keys as location/height and value as temperature values
    (f"event_solid", f"event_solid_mesh", f"event_air_mesh", "event_coeff_mesh")
)